
## Example 1 : Variant of Knapsack Problem

<div class="alert alert-block" style="background-color:lightgray; border-color:black white black white">
You are at a candy store and would like to buy $100$ grams of candies. We have the problem of choosing candies to buy (think of this as the ILP version of the diet problem). We have a bunch of $n$ different varieties of candies $D_1, \ldots, D_n$ that we could choose from.
<ul>
  <li> For each candy $D_j$, we can choose at most $k_j$ candies.
  <li> The cost of one piece of candy type $D_j$ is given by $p_j$.
  <li> The number of Calories/piece in candy type $D_j$ is given by $c_j$.
  <li> We need to minimize the overall cost of our purchase.
  <li> A candy gift box can hold at most $N$ candies and we wish our candies to fit inside a gift box.
  <li> The number of calories must be at least $C_{\min}$ and at most $C_{\max}$.
</ul>
    
 The problem is an integer linear program since we have to choose a whole number of candies. The problem data is given by $n$ the number of candy types, $(k_1, \ldots, k_n)$ how many of each candy type are available for purchase, the prices $(p_1, \ldots, p_n)$, the calories/piece $(c_1, \ldots, c_n)$, the limit $N$ on number of candies per box and caloric limits $C_{\min}$ and $C_{\max}$.
    
The ILP has decision variables
    $$x_1, \ldots, x_n,$$
wherein $x_i$ denotes the number of candies of type $i$ that are to be purchased. Each $x_i \in \mathbb{Z}$ (is an integer) and must satisfy the limits:
    $$ 0 \leq x_i \leq k_i,\ i = 1, \ldots, n $$
Next, the number of candies chosen cannot exceed $N$:
    $$ \sum_{i=1}^n x_i \leq N $$
We must respect the caloric limits:
    $$ C_{\min} \leq \underset{\text{Calories Consumed}}{\underbrace{\sum_{i=1}^n c_i x_i}}  \leq C_{\max} $$

Finally, the objective is to minimize cost. The overall ILP is

$$\begin{array}{rll}
\min& \sum_{j=1}^n p_j x_j & \leftarrow\ \text{minimize total cost of purchase} \\ 
\mathsf{s.t.} & 0 \leq x_j \leq k_j & \leftarrow\ \text{limit on max. number of candies of each type} \\ 
& \sum_{j=1}^n x_j \leq N & \leftarrow\ \text{limit on total number of candies/box}\\ 
& C_{\min} \leq \sum_{i=1}^n c_i x_i  \leq C_{\max}& \leftarrow\ \text{calorie limits} \\ 
& x_1, \ldots, x_n \in \mathbb{Z} & \leftarrow\ \text{integrality constraints}\\
\end{array}$$
</div>

Let's implement this model in python using pulp

In [ ]:
from pulp import * # get all the definitions from pulp library 

def solve_candy_knapsack(n, candy_number_limits, candy_prices, candy_calories, N, Cmin, Cmax):
    assert len(candy_number_limits) == n, 'size mismatch'
    assert len(candy_prices) == n, 'size mismatch for prices'
    assert len(candy_calories) == n, 'size mismatch for list of calories'
    assert N >= 1, 'total number of candies per box must be 1 or more'
    assert Cmin <= Cmax, 'minimum calories is greater than the maximum calories'
    prob = LpProblem('Candy Knapsack', LpMinimize)
    # add decision variables
    # make a list of n decision variables, one for each candy. When declaring the variable, automatically declare
    # its lower bound to be 0 and upper bound to be ki from the candy_number_limits array
    # also declare the category (cat) of the variable to be integers.
    dVars = [LpVariable(f'x{i}',lowBound=0, upBound=ki, cat='Integer') for (i, ki) in enumerate(candy_number_limits)]
    # Now set the objective
    prob += lpSum([pj*xj for (pj,xj) in zip(candy_prices, dVars)])
    # Now add the constraints
    prob += lpSum(dVars) <= N # constraints on number of candies per box
    calories_of_candies = lpSum([cj*xj for (cj,xj) in zip(candy_calories, dVars)])
    prob += calories_of_candies <= Cmax
    prob += calories_of_candies >= Cmin
    status = prob.solve()
    if status == constants.LpStatusInfeasible:
        print('Problem is infeasible')
        return
    elif status == constants.LpStatusUnbounded:
        print('Problem is unbounded. Cannot proceed')
        return
    else:
        assert status == constants.LpStatusOptimal, 'Something went wrong while solving since status is either undefined or unsolved'
        # extract values
        print('Success: optimal answer found')
        solution_values = [x.varValue for x in dVars]
        for (j, svj) in enumerate(solution_values):
            print(f'\t Candy Type # {j}: {svj} candies')
        print(f'Total Cost: {sum([(pj*svj) for (pj, svj) in zip(candy_prices, solution_values)])}')
        print(f'Calories: {sum([cj*xj for (cj,xj) in zip(candy_calories, solution_values)])}')

In [ ]:
n = 5
candy_number_limits = [10, 12, 10, 11, 10]
candy_prices = [0.2, 0.5, 0.1, 0.4, 0.8]
candy_calories = [25, 12, 22, 14, 33]
solve_candy_knapsack(5, candy_number_limits, candy_prices, candy_calories, 12, 250, 500)


Note: you may need to restart the kernel to use updated packages.


In [3]:
from pyomo.environ import *

def solve_candy_knapsack_pyomo(n, candy_number_limits, candy_prices, candy_calories, N, Cmin, Cmax):
    assert len(candy_number_limits) == n, 'size mismatch'
    assert len(candy_prices) == n, 'size mismatch for prices'
    assert len(candy_calories) == n, 'size mismatch for list of calories'
    assert N >= 1, 'total number of candies per box must be 1 or more'
    assert Cmin <= Cmax, 'minimum calories is greater than the maximum calories'

    # Create a model
    model = ConcreteModel()

    # Add decision variables
    model.x = Var(range(n), domain=NonNegativeIntegers, bounds=lambda model, i: (0, candy_number_limits[i]))

    # Objective: Minimize the total cost of candies
    model.obj = Objective(expr=sum(candy_prices[i] * model.x[i] for i in range(n)), sense=minimize)

    # Constraint: Total number of candies per box should be less than or equal to N
    model.candy_limit = Constraint(expr=sum(model.x[i] for i in range(n)) <= N)

    # Constraint: Total calories should be within the range [Cmin, Cmax]
    model.calories_min = Constraint(expr=sum(candy_calories[i] * model.x[i] for i in range(n)) >= Cmin)
    model.calories_max = Constraint(expr=sum(candy_calories[i] * model.x[i] for i in range(n)) <= Cmax)

    # Solve the problem
    solver = SolverFactory('glpk')
    result = solver.solve(model, tee=True)

    # Check the result and print the optimal solution
    if result.solver.status == SolverStatus.ok and result.solver.termination_condition == TerminationCondition.optimal:
        print('Success: optimal answer found')
        solution_values = [model.x[i].value for i in range(n)]
        for j, svj in enumerate(solution_values):
            print(f'\t Candy Type # {j}: {svj} candies')
        total_cost = sum(candy_prices[i] * solution_values[i] for i in range(n))
        total_calories = sum(candy_calories[i] * solution_values[i] for i in range(n))
        print(f'Total Cost: {total_cost}')
        print(f'Calories: {total_calories}')
    else:
        if result.solver.termination_condition == TerminationCondition.infeasible:
            print('Problem is infeasible')
        elif result.solver.termination_condition == TerminationCondition.unbounded:
            print('Problem is unbounded. Cannot proceed')
        else:
            print('Solver status: ', result.solver.status)

# Example usage
n = 3
candy_number_limits = [10, 5, 15]
candy_prices = [1, 2, 3]
candy_calories = [100, 200, 150]
N = 10
Cmin = 500
Cmax = 1500

solve_candy_knapsack_pyomo(n, candy_number_limits, candy_prices, candy_calories, N, Cmin, Cmax)


solver 'glpk'


ApplicationError: No executable found for solver 'glpk'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 kB 942.2 kB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for glpk (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [14 lines of output]
      WARNING setuptools_scm.pyproject_reading toml section missing 'pyproject.toml does not contain a tool.setuptools_scm section'
      Traceback (most recent call last):
        File "/tmp/pip-build-env-8sivbn_8/overlay/lib/python3.8/site-packages/setuptools_scm/_integration/pyproject_reading.py", line 36, in read_pyproject
          section = defn.get("tool", {})[tool_name]
      KeyError: 'setuptools_scm'
      running bdist_wheel
      running build
      running build_ext
      building 'glpk' extension
      creating build
      creating build/temp.linux-x86_64-cpython-38
      creating build/temp.linux-

In [4]:
!pyomo help --solvers



Pyomo Solvers and Solver Managers
---------------------------------
Pyomo uses 'solver managers' to execute 'solvers' that perform
optimization and other forms of model analysis.  A solver directly
executes an optimizer, typically using an executable found on the
user's PATH environment.  Solver managers support a flexible mechanism
for asynchronously executing solvers either locally or remotely.  The
following solver managers are available in Pyomo:

    neos       Asynchronously execute solvers on the NEOS server
    serial     Synchronously execute solvers locally

If no solver manager is specified, Pyomo uses the serial solver
manager to execute solvers locally.  The neos solver manager is used
to execute solvers on the NEOS optimization server.


Serial Solver Interfaces
------------------------
The serial manager supports the following solver interfaces:

    appsi_cbc                     Automated persistent interface to
                                  Cbc
    appsi_cplex    